In [1]:
from scipy import linalg
from scipy import optimize


In [2]:
%run TrainValidTestDataPrep.ipynb

Train Set Normality Test
Valid Set Normality Test
Test  Set Normality Test
3988.HK,(6.738388767369635, 0.03441735335912799)


# story

assume 2018-06-22 (start of validation data) at 02:00 UTC 
due to IOI getting hit/taken we are left with the residual
position as follows:
* +20M HKD 0005.HK
* -5M HKD 0144.HK

As it is convenient to work with weights, I'll convert below
saying that I'm 100% long 0005.HK and 25% short 0144.HK on 
a current notional of 20M HKD

In [3]:
#to standardize where I look up my universe of stocks
univ = data_train.columns

#starting notional
notional = 20e6

#nperiods from post-open to pre-close
kNPERIODS_INTRADAY = 66
kWINDOW = 10 # days
test_date = data_valid.index.date[0]
t0 = data_valid.index[5] # '2018-06-22 02:00:00'
#remaning periods until pre-close from t0
nt = data_valid[(data_valid.index.date == test_date) & (data_valid.index > t0)].shape[0]

In [4]:
# first thing is we need some kind of simple return model so
# we can explore the risk-return in our position, then move to optim

# using sample covar from last 10 days of 5 min returns
cov = data_train[-kWINDOW*kNPERIODS_INTRADAY:].cov()
corr = data_train[-kWINDOW*kNPERIODS_INTRADAY:].corr()

# inspection
var_marginals_1d = pd.Series(np.sqrt(np.diagonal(cov) * kNPERIODS_INTRADAY),index=univ) 
{'whole_intraday_stdev_0005':var_marginals_1d[['0005.HK']],
 'whole_intraday_stdev_0144':var_marginals_1d[['0144.HK']],
 'pairwise_5m_correl':corr.loc['0005.HK','0144.HK']}

{'whole_intraday_stdev_0005': 0005.HK    0.006265
 dtype: float64, 'whole_intraday_stdev_0144': 0144.HK    0.012025
 dtype: float64, 'pairwise_5m_correl': 0.2540967729899665}

In [5]:
# lets assume we can make 30bps each on 0005.HK and 0144.HK if held till pre-close
# note this means -'ve short term expected return on 0144.HK
# and all other names would be E[r] = 0 after TC on the same horizon
er = pd.Series(0,index=univ)
er[['0005.HK','0144.HK']] = [0.003,-0.003]

In [11]:
# check the risk assuming holding from t0 to pre-close
w0 = pd.Series(0,index=univ)
w0[['0005.HK','0144.HK']] = [1.0,-0.25]

def pos_er(w): return np.dot(w,er)
# till end of day 
def pos_var(w): return np.dot(np.dot(w,cov),w) * nt
def pos_pct_stdev(w): return np.sqrt(pos_var(w))
# so the (back of the envenlope) var till end of day
def simple_valatrisk(w): return pos_pct_stdev(w) * 2.0 * notional
def report(w): 
    return {'er': pos_er(w),
            'std': pos_pct_stdev(w),
            'valatrisk':simple_valatrisk(w) / 1e3,
            'pos_long_val':w[w>0.0].sum() * notional / 1e6,
            'pos_short_val':w[w<0.0].sum() * notional / 1e6,
            'max_long':w.max() * notional / 1e6,
            'max_short':w.min() * notional / 1e6}
report(w0)

{'er': 0.00375,
 'std': 0.005932378021069872,
 'valatrisk': 237.2951208427949,
 'pos_long_val': 20.0,
 'pos_short_val': -5.0,
 'max_long': 20.0,
 'max_short': -5.0}

In [7]:
# this is equivalent to unconstrained qp optimization 
# this is the highest risk-variance tradeoff port
# we can make but one look at the var tells us its in 
# need of some constraints/manipulation
wopt1 = np.dot(er,linalg.inv(cov*nt))
report(wopt1)

{'er': 0.518899295670604,
 'std': 0.7203466496559859,
 'valatrisk': 28813.865986239438,
 'pos_long_val': 3356.9188328544496,
 'pos_short_val': -2571.1459348606286,
 'max_long': 2818.7780211911554,
 'max_short': -640.5506166128713}

In [8]:
#one way is to emphasise the covar matrix
#this makes a lot more sane looking portfolio
#the multiplier was fashioned *so as to match the expected return of the origional port*
#but now we have a lower value at risk
wopt2 = np.dot(er,linalg.inv(cov*nt*139))
report(wopt2)

{'er': 0.0037330884580619008,
 'std': 0.0051823499975250835,
 'valatrisk': 207.29399990100336,
 'pos_long_val': 24.15049520039174,
 'pos_short_val': -18.497452768781507,
 'max_long': 20.278978569720547,
 'max_short': -4.608277817358787}

In [9]:
#furthermore we have nearly the same position in our original two names
#so we can make a hedge slice to show back out at IOI or trade
hedge2_trade_list_hkd = ((wopt2 - w0) * notional / 1e6)
hedge2_trade_list_hkd

0001.HK   -0.149027
0002.HK   -0.980035
0003.HK    0.202941
0005.HK    0.278979
0006.HK   -0.495682
0011.HK   -1.437177
0012.HK    0.064948
0016.HK    0.196144
0017.HK   -0.026980
0019.HK   -0.224907
0023.HK   -0.212818
0027.HK   -0.831385
0066.HK   -0.318169
0083.HK    0.158503
0101.HK    0.225505
0144.HK    0.391722
0151.HK    0.218255
0175.HK    0.079240
0267.HK   -0.332007
0288.HK   -0.225376
0386.HK    0.187378
0388.HK   -0.880642
0688.HK    0.794854
0700.HK   -0.327278
0762.HK   -0.463812
0823.HK   -0.993790
0836.HK   -0.176179
0857.HK   -0.541761
0883.HK    0.200466
0939.HK   -1.321200
0941.HK   -0.677884
1038.HK   -0.631795
1044.HK   -0.276557
1088.HK    0.000034
1093.HK   -0.137896
1109.HK   -0.139283
1113.HK   -0.443678
1299.HK   -0.470417
1398.HK    0.328891
1928.HK   -0.301167
1997.HK    0.116268
2007.HK    0.064798
2018.HK    0.574217
2318.HK    0.282657
2319.HK   -0.234639
2382.HK   -0.080092
2388.HK    0.109479
2628.HK   -0.391854
3328.HK    0.066940
3988.HK   -0.165689


In [26]:
#using scipy's minimize function (which is not necessarily the best way to solve with this formulation)
#you can see the solution is almost the same as what I came up with above
er_constraint = ({'type': 'eq', 'fun': lambda w: pos_er(w) - 0.0037330884580619008})
res= optimize.minimize(pos_var, 
                       w0, 
                       method='SLSQP',
                       constraints=er_constraint,
                       options={'maxiter': 100,
                                'ftol':1e-12,
                                'disp':True})
wopt_scipy = res.x
report(wopt_scipy)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 2.6858387213720254e-05
            Iterations: 49
            Function evaluations: 2548
            Gradient evaluations: 49


{'er': 0.0037330884581046747,
 'std': 0.005182507811255112,
 'valatrisk': 207.30031245020447,
 'pos_long_val': 24.12859316167621,
 'pos_short_val': -18.481928697350472,
 'max_long': 20.258707187408326,
 'max_short': -4.628549199956167}

In [28]:
hedge_scipy_trade_list_hkd = ((wopt_scipy - w0) * notional / 1e6)
pd.DataFrame({ 'analytic':hedge2_trade_list_hkd,
               'scipy_SLSQP':hedge_scipy_trade_list_hkd,
               'diff':hedge2_trade_list_hkd  - hedge_scipy_trade_list_hkd}) 

,analytic,scipy_SLSQP,diff
0001.HK,-0.149027,-0.170483,0.021456
0002.HK,-0.980035,-0.949252,-0.030783
0003.HK,0.202941,0.213007,-0.010066
0005.HK,0.278979,0.258707,0.020271
0006.HK,-0.495682,-0.509595,0.013913
0011.HK,-1.437177,-1.447064,0.009887
0012.HK,0.064948,0.065119,-0.000171
0016.HK,0.196144,0.176112,0.020032
0017.HK,-0.026980,-0.027751,0.000771
0019.HK,-0.224907,-0.217097,-0.007810


In [41]:
#so lets go back to the analytical version 
#and see what happens when we double the specific variance
#of one name
#above it was suggested to short 1.4M HKD of 0011.HK
#but lets say due to analyst ratings changes or something 
#we forecast a spike in marginal variance.
cov2 = cov.copy()
cov2.loc['0011.HK','0011.HK'] = cov2.loc['0011.HK','0011.HK'] * 2.0

wopt3 = np.dot(er,linalg.inv(cov2*nt*138))
report(wopt3)

{'er': 0.003728163597378879,
 'std': 0.005188501416936536,
 'valatrisk': 207.54005667746145,
 'pos_long_val': 23.920911599200146,
 'pos_short_val': -18.197033440156567,
 'max_long': 20.23218808573289,
 'max_short': -4.622235896792973}

In [43]:
# now the optmizer shrinks the position in that name, others are unchanged
# no surprise really, but if you have worked with constrained optim much
# this adjustment has a pretty nice localized effect!
hedge3_trade_list_hkd = ((wopt3 - w0) * notional / 1e6)
pd.DataFrame({ 'analytic2':hedge2_trade_list_hkd,
               'analytic3':hedge3_trade_list_hkd,
               'diff':hedge2_trade_list_hkd  - hedge3_trade_list_hkd}) 

,analytic2,analytic3,diff
0001.HK,-0.149027,-0.295882,0.146855
0002.HK,-0.980035,-1.000915,0.020880
0003.HK,0.202941,0.216255,-0.013315
0005.HK,0.278979,0.232188,0.046790
0006.HK,-0.495682,-0.528105,0.032423
0011.HK,-1.437177,-0.594321,-0.842856
0012.HK,0.064948,0.007337,0.057611
0016.HK,0.196144,0.122116,0.074028
0017.HK,-0.026980,-0.005289,-0.021691
0019.HK,-0.224907,-0.271759,0.046851


In [ ]:
#todo: show the opposite of the above - the case where cross sectional effects are enhanced or weakened